## Different neighbourhood Venues in Toronto Downtown and Manhattan¶

### Background

### Datas

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [4]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


#### All the Borough,Neighborhood,Latitude,Longitude datas are extracted for New York
                                         
                                         
                                          

In [43]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
                                                                   

neighborhoods.head()


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [44]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)
neighborhoods.shape

The dataframe has 5 boroughs and 612 neighborhoods.


(612, 4)

#### Now lets extract the Neighborhoods only for Manhattan

In [45]:
Manhattan_data=neighborhoods[neighborhoods["Borough"]=="Manhattan"].reset_index(drop=True)
Manhattan_data["City"]="NY"
Manhattan_data.head(5)

,Borough,Neighborhood,Latitude,Longitude,City
0,Manhattan,Marble Hill,40.876551,-73.910660,NY
1,Manhattan,Chinatown,40.715618,-73.994279,NY
2,Manhattan,Washington Heights,40.851903,-73.936900,NY
3,Manhattan,Inwood,40.867684,-73.921210,NY
4,Manhattan,Hamilton Heights,40.823604,-73.949688,NY


In [49]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7896239, -73.9598939.


#### Visualizing neighbourhoods of Manhattan

In [50]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Manhattan_data['Latitude'], Manhattan_data['Longitude'], Manhattan_data['Borough'],Manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [9]:
VERSION = '20180605' # Foursquare API version
CLIENT_ID="Y52AQ2NITZ41GV1SDPN0W2OSE5I5QXF2ZQ4XFPYYNRLRUEKW"
CLIENT_SECRET="MHJI3KRQLHDT4HUWGZAFZCPI3Z4QXAJLBLLNX4LKF2YEOBRU"
LIMIT=100
radius=500
print("CLIENT_ID" +"="+ CLIENT_ID +"CLIENT_SECRET"+ "=" + CLIENT_SECRET,"VERSION"+"="+VERSION)

CLIENT_ID=Y52AQ2NITZ41GV1SDPN0W2OSE5I5QXF2ZQ4XFPYYNRLRUEKWCLIENT_SECRET=MHJI3KRQLHDT4HUWGZAFZCPI3Z4QXAJLBLLNX4LKF2YEOBRU VERSION=20180605


#### All the Borough,Neighborhood,Latitude,Longitude datas are extracted for Toronto

In [14]:
import requests
!wget -q -O 'Toronto_CAN.json' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M       
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
dfs=pd.read_html(website_url)
d=dfs[0]
d.head(5)
        

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### cleansing the data

In [15]:
df1 = d.dropna(axis=0)
df1.reset_index(inplace=True)
df1.drop('index', axis=1,inplace=True)
df1.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df1['Borough'].unique()),
        df1.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


#### final clean data containing Latitude and longitude for Toronto

In [51]:
geo=pd.read_csv("http://cocl.us/Geospatial_data")

#geo.head()
p_code_df1=df1["Postal Code"].values
p_code_geo=geo["Postal Code"].values

Lat=[]
Long=[]
for i in p_code_df1:
    for j in p_code_geo:
        if(i==j):
            idx=geo[geo["Postal Code"]==j].index.values[0]
            Lat.append(geo.ix[idx,1])
            Long.append(geo.ix[idx,2])

df2 = pd.DataFrame(Lat,columns =['Latitude'])
df3 = pd.DataFrame(Long,columns =['Longitude'])
frames = [df1,df2,df3]
result = pd.concat(frames,axis=1)
result["City"] = "Tor"
result.head(5)  

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,Postal Code,Borough,Neighborhood,Latitude,Longitude,City
0,M3A,North York,Parkwoods,43.753259,-79.329656,Tor
1,M4A,North York,Victoria Village,43.725882,-79.315572,Tor
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,Tor
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,Tor
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,Tor


#### Neighborhoods of Downtown Toronto alone is extrated

In [54]:
Downtown_Toronto_data=result[result["Borough"]=="Downtown Toronto"].reset_index(drop=True)
Downtown_Toronto_data.drop(["Postal Code"],axis = 1,inplace=True)
Downtown_Toronto_data.head(5)

,Borough,Neighborhood,Latitude,Longitude,City
0,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,Tor
1,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,Tor
2,Downtown Toronto,Garden District / Ryerson,43.657162,-79.378937,Tor
3,Downtown Toronto,St. James Town,43.651494,-79.375418,Tor
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,Tor


In [18]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Visualizing downtown Toronto

In [56]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Downtown_Toronto_data["Latitude"], Downtown_Toronto_data["Longitude"], Downtown_Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Merge both Toronto and Manhattan datas

In [57]:

Tot_data = pd.concat([Manhattan_data, Downtown_Toronto_data]).reset_index(drop=True)

Tot_data.head(5)


,Borough,Neighborhood,Latitude,Longitude,City
0,Manhattan,Marble Hill,40.876551,-73.910660,NY
1,Manhattan,Chinatown,40.715618,-73.994279,NY
2,Manhattan,Washington Heights,40.851903,-73.936900,NY
3,Manhattan,Inwood,40.867684,-73.921210,NY
4,Manhattan,Hamilton Heights,40.823604,-73.949688,NY


In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### All venues of sorrounding neighborhood for both Toronto and Manhattan is outputted

In [58]:
All_venues = getNearbyVenues(names=Tot_data['Neighborhood'],
                                          latitudes=Tot_data['Latitude'],
                                          longitudes=Tot_data['Longitude']
                                          )
All_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


#### Analyze each Neighborhood

In [59]:
# one hot encoding
All_venues_onehot = pd.get_dummies(All_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
All_venues_onehot['Neighborhood'] = All_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [All_venues_onehot.columns[-1]] + list(All_venues_onehot.columns[:-1])
All_venues_onehot = All_venues_onehot[fixed_columns]

All_venues_onehot.shape

(7201, 358)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [60]:
All_venues_onehot_grouped = All_venues_onehot.groupby('Neighborhood').mean().reset_index()
All_venues_onehot_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Public Art,Ramen Restaurant,R

In [61]:
All_venues_onehot_grouped.shape

(59, 358)

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 5 venues for each neighborhood.

In [75]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = All_venues_onehot_grouped['Neighborhood']

for ind in np.arange(All_venues_onehot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(All_venues_onehot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Battery Park City,Park,Hotel,Memorial Site,Gym,Coffee Shop
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Café
2,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport Terminal,Bar,Plane
3,Carnegie Hill,Coffee Shop,Yoga Studio,Bakery,Japanese Restaurant,Bookstore
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Bar


#### Cluster Neighborhoods

In [71]:
# set number of clusters
kclusters = 4

All_venues_onehot_grouped_clustering = All_venues_onehot_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(All_venues_onehot_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

array([1, 3, 1, 3, 3, 1, 3, 1, 3, 3, 1, 1, 3, 1, 1, 3, 3, 1, 3, 1, 1, 3,
       3, 1, 1, 3, 1, 1, 1, 1, 1, 3, 3, 1, 1, 3, 1, 1, 3, 3, 3, 3, 2, 1,
       3, 3, 3, 0, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [65]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Tor_NY_merged = Tot_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Tor_NY_merged = Tor_NY_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Tor_NY_merged.head() # check the last columns!


(99, 11)

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Tor_NY_merged['Latitude'], Tor_NY_merged['Longitude'], Tor_NY_merged['Neighborhood'], Tor_NY_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [66]:
Tor_NY_merged.loc[Tor_NY_merged['Cluster Labels'] == 0, Tor_NY_merged.columns[[1] +[4]+ list(range(5, Tor_NY_merged.shape[1]))]]


,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,Stuyvesant Town,NY,0,Boat or Ferry,Park,Farmers Market,Pet Service,Baseball Field
77,Stuyvesant Town,NY,0,Boat or Ferry,Park,Farmers Market,Pet Service,Baseball Field


In [67]:
Tor_NY_merged.loc[Tor_NY_merged['Cluster Labels'] == 1, Tor_NY_merged.columns[[1] +[4]+ list(range(5, Tor_NY_merged.shape[1]))]]


,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Chinatown,NY,1,Chinese Restaurant,Bakery,Cocktail Bar,Coffee Shop,Optical Shop
2,Washington Heights,NY,1,Café,Pizza Place,Bakery,Chinese Restaurant,Grocery Store
3,Inwood,NY,1,Mexican Restaurant,Pizza Place,Café,Bakery,Restaurant
6,Central Harlem,NY,1,African Restaurant,American Restaurant,Seafood Restaurant,Cosmetics Shop,Bar
7,East Harlem,NY,1,Mexican Restaurant,Bakery,Thai Restaurant,Latin American Restaurant,Deli / Bodega
8,Upper East Side,NY,1,Italian Restaurant,Bakery,Juice Bar,Gym / Fitness Center,Exhibit
9,Yorkville,NY,1,Coffee Shop,Italian Restaurant,Gym,Deli / Bodega,Bar
10,Lenox Hill,NY,1,Italian Restaurant,Coffee Shop,Pizza Place,Sushi Restaurant,Cocktail Bar
12,Upper West Side,NY,1,Italian Restaurant,Bakery,Wine Bar,Coffee Shop,Middle Eastern Restaurant
13,Lincoln Square,NY,1,Café,Plaza,Concert Hall,Gym / Fitness Center,Theater


In [68]:
Tor_NY_merged.loc[Tor_NY_merged['Cluster Labels'] == 2, Tor_NY_merged.columns[[1] +[4]+ list(range(5, Tor_NY_merged.shape[1]))]]


,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
94,Rosedale,Tor,2,Park,Playground,Trail,Dry Cleaner,Discount Store


In [69]:
Tor_NY_merged.loc[Tor_NY_merged['Cluster Labels'] == 3, Tor_NY_merged.columns[[1] +[4]+ list(range(5, Tor_NY_merged.shape[1]))]]


,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Marble Hill,NY,3,Sandwich Place,Coffee Shop,Gym,American Restaurant,Yoga Studio
4,Hamilton Heights,NY,3,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant
5,Manhattanville,NY,3,Coffee Shop,Seafood Restaurant,Italian Restaurant,Park,Mexican Restaurant
11,Roosevelt Island,NY,3,Park,Playground,Bubble Tea Shop,Farmers Market,Dry Cleaner
17,Chelsea,NY,3,Art Gallery,Coffee Shop,Ice Cream Shop,Italian Restaurant,Juice Bar
26,Morningside Heights,NY,3,Park,Bookstore,Coffee Shop,American Restaurant,Deli / Bodega
29,Financial District,NY,3,Coffee Shop,Hotel,Pizza Place,Café,Juice Bar
30,Carnegie Hill,NY,3,Coffee Shop,Yoga Studio,Bakery,Japanese Restaurant,Bookstore
40,Marble Hill,NY,3,Sandwich Place,Coffee Shop,Gym,American Restaurant,Yoga Studio
44,Hamilton Heights,NY,3,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant


### Observation

### Discussions

### Conclusion